In [1]:
def coc():
    dfc1=pd.DataFrame()
    headers = {'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    req = requests.get("https://www.oyorooms.com/hotels-in-kochi/", headers=headers).text
    soup = Soup(req, 'html.parser')
    apts = soup.find_all("div", {"class": "oyo-row oyo-row--no-spacing listingHotelDescription"})
    l = []
    obj = {}
    for a in range(0, len(apts)):
        try:
            obj["pricing"] = apts[a].find("span", {"class": "listingPrice__finalPrice"}).text
        except: obj["pricing"] = None
        try:
            obj["address"] = apts[a].find("div", {"class": "d-body-lg listingHotelDescription__hotelAddress"}).text
        except:
            obj["address"] = None
        l.append(obj)
        obj = {}
    for i in l:
        price = i["pricing"]
        address = i["address"]
        dfc1 = dfc1.append({"price": price,"address": address}, ignore_index=True)
    dfc2=pd.DataFrame()
    for page in range(1,14):
        req1 = requests.get("https://www.booking.com/searchresults.en-us.html?ss=Cochin&ssne=Cochin&ssne_untouched=Cochin&label=gog235jc-1DCAMobEIGY29jaGluSDNYA2hsiAEBmAExuAEXyAEM2AED6AEB-AECiAIBqAIDuAKryf-hBsACAdICJGNmMmQ1ODhmLTJiYWYtNGQyNS05YzE5LWM2OWQyZjk4OTNmNdgCBOACAQ&sid=98568152db64b2f303d5bb990da98b4c&aid=356980&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-2093645&dest_type=city&checkin=2023-05-01&checkout=2023-05-02&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure/{}_p/".format(page),headers=headers).text
        soup = Soup(req1, 'html.parser')
        apts = soup.find_all("div", {"class": "a1b3f50dcd a1f3ecff04 f7c6687c3d f996d8c258"})
        l = []
        obj = {}
        for a in range(0, len(apts)):
            try:
                obj["pricing"] = apts[a].find("span", {"class": "fcab3ed991 fbd1d3018c e729ed5ab6"}).text
            except: obj["pricing"] = None
            try:
                obj["address"] = apts[a].find("span", {"class": "f4bd0794db b4273d69aa"}).text
            except:
                obj["address"] = None
            l.append(obj)
            obj = {}
        for i in l:
            price = i["pricing"]
            address = i["address"]
            dfc2 = dfc2.append({"price": price,"address": address}, ignore_index=True)
    df = pd.concat([dfc1, dfc2], ignore_index=True)
    df["price"] = df["price"].str.replace(r"₹", "")
    df["price"] = df["price"].str.replace(r",", "")
    df['price'] = df['price'].astype(float)
    return df
coc()

NameError: name 'pd' is not defined

In [ ]:
#added more features
import requests 
from bs4 import BeautifulSoup as Soup
import pandas as pd
def delhi():
    df=pd.DataFrame()
    headers = {'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    for page in range (1,6):
        req = requests.get("https://www.oyorooms.com/hotels-in-delhi/", headers=headers).text
        soup = Soup(req, 'html.parser')
        apts = soup.find_all("div", {"class": "oyo-row oyo-row--no-spacing listingHotelDescription"})
        l = []
        obj = {}
        apts1 = soup.find_all(itemprop='url')
        for a in range(0, len(apts)):
            try:
                obj["pricing"] = apts[a].find("span", {"class": "listingPrice__finalPrice"}).text
            except: obj["pricing"] = None
            try:
                obj["Distance"] = apts[a].find("span", {"class": "listingHotelDescription__distanceText"}).text
            except:
                obj["Distance"] = None
            try:
                link= apts1[a].get('content')
                req1 = requests.get(link, headers=headers).text
                soup2 = Soup(req1, 'html.parser')
                obj["amneties"] = soup2.find("div", {"class": "c-2cojj"}).text
            except:
                obj["amneties"] = None
            try:
                obj["ratings"] = apts[a].find("div", {"class": "hotelRating"}).text
            except:
                obj["ratings"] = None
            
            l.append(obj)
            obj = {}
        for i in l:
            price = i["pricing"]
            distance = i["Distance"]
            amneties= i["amneties"]
            ratings= i["ratings"]
            df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
#dataCleaning
    df["price"] = df["price"].str.replace(r"₹", "")
    df["ratings"] = df["ratings"].str.replace(r"₹", "")
    df['price'] = pd.to_numeric(df['price'])
    df['distance']=df['distance'].str.replace(r"km","")
    df['distance'] = pd.to_numeric(df['distance'])
    df[['ratings', 'last_name','y','x']] = df['ratings'].str.split(' ', expand=True)
    df = df.drop(columns=['last_name','y','x'],axis=1)
    df['ratings'] = pd.to_numeric(df['ratings'],errors='coerce')
    import re
    df['wifi'] = df['amneties'].str.extract(r'(Wifi)', expand=False,flags=re.IGNORECASE)
    df['AC'] = df['amneties'].str.extract(r'(AC)', expand=False)
    df['amneties']=df['amneties'].str.replace(r"CCTV","")
    df['TV'] = df['amneties'].str.extract(r'(TV)', expand=False)
    df['Queen Sized Bed'] = df['amneties'].str.extract(r'(Queen Sized Bed)', expand=False)
    df['Geyser'] = df['amneties'].str.extract(r'(Geyser)', expand=False)
    df['King Sized Bed'] = df['amneties'].str.extract(r'(King Sized Bed)', expand=False)
    df['Power backup'] = df['amneties'].str.extract(r'(Power backup)', expand=False)
    df['Mini Fridge'] = df['amneties'].str.extract(r'(Mini Fridge)', expand=False)
    df= df.drop(columns=['amneties'],axis= 1)
    df['AC'] = df['AC'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['TV'] = df['TV'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['wifi'] = df['wifi'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['Queen Sized Bed'] = df['Queen Sized Bed'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['King Sized Bed'] = df['King Sized Bed'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['Geyser'] = df['Geyser'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['Power backup'] = df['Power backup'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['Mini Fridge'] = df['Mini Fridge'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['Mini Fridge'] = df['Mini Fridge'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    df['ratings'] = df['ratings'].fillna(df['ratings'].mean())
    return df
df= delhi()
df

C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\7834177

C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\7834177

C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\783417779.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
C:\Users\Hashif\AppData\Local\Temp\ipykernel_26400\7834177

,price,distance,ratings,wifi,AC,TV,Queen Sized Bed,Geyser,King Sized Bed,Power backup,Mini Fridge
0,799,5.0,4.5,1,1,1,1,0,0,1,0
1,699,1.4,3.5,1,1,1,0,1,0,1,0
2,576,6.5,3.0,1,1,1,0,0,0,0,0
3,999,4.6,2.8,1,1,1,0,1,0,1,0
4,999,3.6,3.7,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
95,727,3.7,3.5,1,1,1,0,1,0,1,0
96,599,4.6,3.9,1,1,1,0,1,0,1,0
97,1782,2.8,4.1,1,1,1,0,0,0,0,0
98,576,8.5,2.4,1,1,1,0,1,0,1,0


,price,distance,amneties,ratings
0,799,5.0,ACWiFiQueen Sized BedTVPower backupReception,4.5
1,499,3.9,ACTVFree WifiFully Functional KitchenKitchenFr...,3.2
2,626,2.0,ACFree WifiTVGeyserPower backupElevator,2.9
3,576,6.5,WiFiACTV,3.0
4,699,1.4,ACFree WifiTVPower backupGeyserElevator,3.5
...,...,...,...,...
95,796,4.5,ACFree WifiTVGeyserPower backup,NEW
96,876,4.5,ACTVFree WifiKing Sized BedMini FridgeSeating ...,2.8
97,1499,5.0,ACTVFree WifiGeyserPower backupElevator,4.0
98,576,8.5,ACFree WifiTVGeyserPower backup,2.4


In [ ]:
import requests 
from bs4 import BeautifulSoup as Soup
import pandas as pd
def delhi():
    df=pd.DataFrame()
    headers = {'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    req = requests.get("https://www.oyorooms.com/hotels-in-delhi/", headers=headers).text
    soup = Soup(req, 'html.parser')
    apts = soup.find_all("div", {"class": "oyo-row oyo-row--no-spacing listingHotelDescription"})
    l = []
    obj = {}
    apts1 = soup.find_all(itemprop='url')
    for a in range(0, len(apts)):
        try:
            obj["pricing"] = apts[a].find("span", {"class": "listingPrice__finalPrice"}).text
        except: obj["pricing"] = None
        try:
            obj["Distance"] = apts[a].find("span", {"class": "listingHotelDescription__distanceText"}).text
        except:
            obj["Distance"] = None
        try:
            link= apts1[a].get('content')
            req1 = requests.get(link, headers=headers).text
            soup2 = Soup(req1, 'html.parser')
            obj["amneties"] = soup2.find("div", {"class": "c-2cojj"}).text
        except:
            obj["amneties"] = None
        try:
            obj["ratings"] = apts[a].find("div", {"class": "hotelRating"}).text
        except:
            obj["ratings"] = None
            
        l.append(obj)
        obj = {}
    for i in l:
        price = i["pricing"]
        distance = i["Distance"]
        amneties= i["amneties"]
        ratings= i["ratings"]
        df = df.append({"price": price,"distance": distance,"amneties":amneties,"ratings":ratings}, ignore_index=True)
    df["price"] = df["price"].str.replace(r"₹", "")
    df["ratings"] = df["ratings"].str.replace(r"₹", "")
    df['price'] = pd.to_numeric(df['price'])
    df['distance']=df['distance'].str.replace(r"km","")
    df['distance'] = pd.to_numeric(df['distance'])
    df[['ratings', 'last_name','y','x']] = df['ratings'].str.split(' ', expand=True)
    df = df.drop(columns=['last_name','y','x'],axis=1)
    df['ratings'] = pd.to_numeric(df['ratings'],errors='coerce')
    dfg['AC'] = dfg['AC'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['TV'] = dfg['TV'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['wifi'] = dfg['wifi'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['Queen Sized Bed'] = dfg['Queen Sized Bed'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['King Sized Bed'] = dfg['King Sized Bed'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['Geyser'] = dfg['Geyser'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['Power backup'] = dfg['Power backup'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['Mini Fridge'] = dfg['Mini Fridge'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    dfg['Mini Fridge'] = dfg['Mini Fridge'].fillna('').replace({'': 0, '[^0]': 1}, regex=True)
    import numpy as np
    dfg['ratings'] = df['ratings'].replace({'NEW': np.nan})
    df['ratings'] = pd.to_numeric(df['ratings'],errors='coerce')
    dfg['ratings'] = dfg['ratings'].fillna(df['ratings'].mean())
    dfg

    return df
dfd= delhi()

,price,distance,ratings,wifi,AC,TV,Queen Sized Bed,Geyser,King Sized Bed,Power backup,Mini Fridge
0,799,5.0,4.5,WiFi,AC,TV,Queen Sized Bed,NaN,NaN,Power backup,NaN
1,499,3.9,3.2,Wifi,AC,TV,NaN,NaN,NaN,NaN,NaN
2,626,2.0,2.9,Wifi,AC,TV,NaN,Geyser,NaN,Power backup,NaN
3,576,6.5,3.0,WiFi,AC,TV,NaN,NaN,NaN,NaN,NaN
4,699,1.4,3.5,Wifi,AC,TV,NaN,Geyser,NaN,Power backup,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,796,4.5,NEW,Wifi,AC,TV,NaN,Geyser,NaN,Power backup,NaN
96,876,4.5,2.8,Wifi,AC,TV,NaN,NaN,King Sized Bed,NaN,Mini Fridge
97,1499,5.0,4.0,Wifi,AC,TV,NaN,Geyser,NaN,Power backup,NaN
98,576,8.5,2.4,Wifi,AC,TV,NaN,Geyser,NaN,Power backup,NaN


,price,distance,ratings,wifi,AC,TV,Queen Sized Bed,Geyser,King Sized Bed,Power backup,Mini Fridge
0,799,5.0,4.5,1,1,1,1,0,0,1,0
1,499,3.9,3.2,1,1,1,0,0,0,0,0
2,626,2.0,2.9,1,1,1,0,1,0,1,0
3,576,6.5,3.0,1,1,1,0,0,0,0,0
4,699,1.4,3.5,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
95,796,4.5,NEW,1,1,1,0,1,0,1,0
96,876,4.5,2.8,1,1,1,0,0,1,0,1
97,1499,5.0,4.0,1,1,1,0,1,0,1,0
98,576,8.5,2.4,1,1,1,0,1,0,1,0


,price,distance,ratings,wifi,AC,TV,Queen Sized Bed,Geyser,King Sized Bed,Power backup,Mini Fridge
0,799,5.0,4.5,1,1,1,1,0,0,1,0
1,499,3.9,3.2,1,1,1,0,0,0,0,0
2,626,2.0,2.9,1,1,1,0,1,0,1,0
3,576,6.5,3.0,1,1,1,0,0,0,0,0
4,699,1.4,3.5,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
95,796,4.5,3.526316,1,1,1,0,1,0,1,0
96,876,4.5,2.8,1,1,1,0,0,1,0,1
97,1499,5.0,4.0,1,1,1,0,1,0,1,0
98,576,8.5,2.4,1,1,1,0,1,0,1,0


In [ ]:
import requests 
from bs4 import BeautifulSoup as Soup
import pandas as pd
df=pd.DataFrame()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
req = requests.get("https://www.makemytrip.com/hotels/hotel-listing/?checkin=04252023&city=CTGOI&checkout=04262023&roomStayQualifier=3e0e&locusId=CTGOI&country=IN&locusType=city&searchText=Goa,%20India&regionNearByExp=3", headers=headers).status_code
req

200